In [1]:
import os
import sys
import json
import re

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import custom_cde as cde
from custom_cde import Document
from custom_cde.doc import Paragraph, Heading
# from custom_cde.model import Compound
# from custom_cde.model import BaseModel, StringType, ListType, ModelType
# from custom_cde.parse import R, I, W, Optional, merge
# from custom_cde.parse.base import BaseParser
# from custom_cde.utils import first

%load_ext autoreload

In [13]:
d = Document(
#     Heading('Molecular energy levels of poly(3-hexylthiophene) (P3HT)'),
#     Paragraph('P3HT has a HOMO level of -4.9 eV, while the LUMO level is -3.0 eV. This corresponds to a bandgap of 1.9 eV.'),
#     Paragraph('For P3HT, EHOMO is -4.9 to -5.2 eV'),
#     Paragraph('For P3HT, ELUMO is -3.0 to -2.9 eV'),
#     Paragraph('for P3ht, the EBandGap is 1.9 to 2.3 eV'),
    
#     #what about the following sentence doesn't work, compared to the next one?
#     Paragraph(u'The gold electrode has a Fermi energy level of 5.1 eV'),
#     Paragraph(u'For gold, the Fermi energy level = 5.1 eV'),
    
#     Heading('Synthesis of 2,4,6-trinitrotoluene (3a)'),
#     Paragraph('The procedure was followed to yield a pale yellow solid (b.p. 240 °C)'),
    
    Heading('New property phrases'),
#     Paragraph('the Tm of gold was found to be ~1600 °C.'),
#     Paragraph('The melting point of P3HT is 275 ºC.'),
#     Paragraph('The Tm of P3HT is 275 ºC.'),
#     Paragraph('the boiling point of niacin is 100 ºC.'),
#     Paragraph('For niacinamide, the corrosion inhibition efficiency = 90%.'),
    Paragraph('for P3HT, the enthalpy of sublimation is 100 kJ/g'),
    Paragraph('for P3HT, the enthalpy of vaporization is 73 J/g'),
    Paragraph('for P3HT, the enthalpy of crystallization is 27 J/g'),
)
d

<Document: 4 elements>

In [15]:
%autoreload

d.records.serialize()

[{'enthalpy_of_fusion': [{'value': '100', 'units': 'kJ/g'}]},
 {'enthalpy_of_vaporization': [{'value': '100', 'units': 'kJ/g'}]},
 {'enthalpy_of_sublimation': [{'value': '100', 'units': 'kJ/g'}]},
 {'enthalpy_of_fusion': [{'value': '73', 'units': 'J/g'}]},
 {'enthalpy_of_vaporization': [{'value': '73', 'units': 'J/g'}]},
 {'enthalpy_of_sublimation': [{'value': '73', 'units': 'J/g'}]},
 {'enthalpy_of_fusion': [{'value': '27', 'units': 'J/g'}]},
 {'enthalpy_of_vaporization': [{'value': '27', 'units': 'J/g'}]},
 {'enthalpy_of_sublimation': [{'value': '27', 'units': 'J/g'}]},
 {'names': ['P3HT']}]